# CME 250 Machine Learning Project 2
## -- Sweekar Bengaluru

Data Source: The data for this project is obtained from https://www.kaggle.com/competitions/cme250-final-project/data


### Introduction
The project is on using Machine learning algorithms to analyse the patient data and predict whether the patient would progress to severe dengue case or not. Algorithm is built to correlate over 3500 gene signatures from 300 observations to predict if the patient progresses to severe case of dengue. 

The project report is sumitted on kaggle under the username: s.sweekar-


### Prediction algorithm development:
Since this is a Supervised learning and a classification problem, Logistic regression, Random forest and neural net algorithms were tried out. Initially, a simple logistic regression algorithm was built and then dimension reduction techniques, cross validation techniques were applied. In addition, different solver and regularisation/penalisation methods were explored. lbfgs, newton-cg, sag, saga, liblinear solver models were tried, L1 and L2 regularisation technique was also applied. For the given dataset, with iteration count of 1000, lbfgs, C=2.5(Regularisation metric, smaller the value, higher the extent of regularisation) turned out to provide best results during validation.


In [3]:
#Import required packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [8]:
#Reading data and checking data quality
data = pd.read_csv("train1.csv",index_col = "SampleID")
data.head()
X = data.drop(['class','dataset'],axis='columns')
y = data['class']

for col in X.columns:
    if X[col].isnull().sum()/len(X)>0:
        print(col)

In [9]:
#Data split and logistic regression application
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=0)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
clf = LogisticRegression(penalty='l2',solver='lbfgs',C=2.5,max_iter=2500,random_state=0).fit(X_train, y_train)
y_val_logit_hat = clf.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_val_logit_hat, pos_label=1)
auc_logit = metrics.auc(fpr, tpr)
print(auc_logit)
print("Train accuracy:", np.round(accuracy_score(y_train, 
                                                 clf.predict(X_train)), 2))
print("Test accuracy:", np.round(accuracy_score(y_test, y_val_logit_hat), 2))
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,clf.predict(X_test)))
print(classification_report(y_test,clf.predict(X_test)))


0.7554593616746349
Train accuracy: 1.0
Test accuracy: 0.77
[[91 18]
 [23 48]]
              precision    recall  f1-score   support

           0       0.80      0.83      0.82       109
           1       0.73      0.68      0.70        71

    accuracy                           0.77       180
   macro avg       0.76      0.76      0.76       180
weighted avg       0.77      0.77      0.77       180



The recall and precision results are reasonable. These are centered around 0.7.

### Prediction

In [10]:
data_test = pd.read_csv("test1.csv",index_col = "SampleID")
data_test.head()
X_test_final = data_test.drop(['dataset'],axis='columns')
y_predict = pd.DataFrame(clf.predict_proba(X_test_final),columns=clf.classes_)[1]
y_pred_df = pd.DataFrame(y_predict)
y_pred_df.columns = ["class"]
y_pred_df.index = data_test.index
y_pred_df.to_csv("PCA_Logistic_Benchmark_8.csv")

With this code, the AUC on the unseen data turned out to be 0.77. In next iteration, Dimension reduction techniques were applied and it was found that 17 dimensions were required to accurately represent data. With this reduced dimension and cross validation, AUC on unseen data turned out to be 0.45 which was surprising!


In next iteration, PCA and cross validation was used, but the logistic regression model was trained on the entire data and the AUC for unseen data turned out to be 0.8.
Similarly, random forest and tree pruning techniques were applied and results for AUC for unseen data was around 0.6. 


Then a neural net model was trained on the entire data using MLP classifier available in Scikit learn. Different neural net architectures were tried and for varying depth and breadth. The best results were centered arouund 0.6 AUC for unseen data

Finally, referencing paper, 8 significant gene signatures were identified and this was used to train logistic regression and MLP classifier (Neural net model with 7,7,7). This returned the results of 0.77 and 0.76 respectively on the test data.

### Improvement
Possile improvement could be on iterating on the neural net architecture so as to improve prediction, reduce overfitting. Data cleanup would also help by removing erranoeus and outlier datapoints.